In [1]:
# !pip install pycaret --user 
# requires new python environment to be created for 3.9
# !pip install git+https://github.com/pycaret/pycaret.git#egg=pycaret

  Cloning https://github.com/pycaret/pycaret.git to c:\users\madalina.ciortan\appdata\local\temp\pip-install-ro2dp_wp\pycaret_5475978a352d4fe4ac34f756c58a2bbc
  Resolved https://github.com/pycaret/pycaret.git to commit 885ebb81055ac017e50080db7731973d1a222e19
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ipywidgets-8.1.2-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 217.9 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.0 kB 217.9 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.0 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 325.6 kB/s eta 0:00:00
  Using cached jinja2-3.1.4-py3-none-any.whl.

  Running command git clone --filter=blob:none --quiet https://github.com/pycaret/pycaret.git 'C:\Users\madalina.ciortan\AppData\Local\Temp\pip-install-ro2dp_wp\pycaret_5475978a352d4fe4ac34f756c58a2bbc'


https://www.pycaret.org/tutorials/html/ANO101.html

In [4]:
import pycaret
from pycaret.anomaly import *
import pandas as pd

In [6]:

data = pd.read_csv('https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/nyc_taxi.csv')

data['timestamp'] = pd.to_datetime(data['timestamp'])

data.head()

,timestamp,value
0,2014-07-01 00:00:00,10844
1,2014-07-01 00:30:00,8127
2,2014-07-01 01:00:00,6210
3,2014-07-01 01:30:00,4656
4,2014-07-01 02:00:00,3820


In [7]:
data['MA48'] = data['value'].rolling(48).mean()
data['MA336'] = data['value'].rolling(336).mean()

# plot 
import plotly.express as px
fig = px.line(data, x="timestamp", y=['value', 'MA48', 'MA336'], title='NYC Taxi Trips', template = 'plotly_dark')
fig.show()

In [8]:
# drop moving-average columns
data.drop(['MA48', 'MA336'], axis=1, inplace=True)

# set timestamp to index**
data.set_index('timestamp', drop=True, inplace=True)

# resample timeseries to hourly **
data = data.resample('H').sum()

# creature features from date**
data['day'] = [i.day for i in data.index]
data['day_name'] = [i.day_name() for i in data.index]
data['day_of_year'] = [i.dayofyear for i in data.index]
data['week_of_year'] = [i.weekofyear for i in data.index]
data['hour'] = [i.hour for i in data.index]
data['is_weekday'] = [i.isoweekday() for i in data.index]

data.head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday
timestamp,,,,,,,
2014-07-01 00:00:00,18971,1,Tuesday,182,27,0,2
2014-07-01 01:00:00,10866,1,Tuesday,182,27,1,2
2014-07-01 02:00:00,6693,1,Tuesday,182,27,2,2
2014-07-01 03:00:00,4433,1,Tuesday,182,27,3,2
2014-07-01 04:00:00,4379,1,Tuesday,182,27,4,2


In [9]:
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(5160, 7)"
2,Transformed data shape,"(5160, 13)"
3,Numeric features,6
4,Categorical features,1
5,Preprocess,True
6,Imputation type,simple
7,Numeric imputation,mean
8,Categorical imputation,mode
9,Maximum one-hot encoding,-1


In [10]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pycaret.internal.patches.pyod.CBLOFForceToDouble
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [12]:
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2014-07-01 00:00:00,18971,1,Tuesday,182,27,0,2,0,-0.013313
2014-07-01 01:00:00,10866,1,Tuesday,182,27,1,2,0,-0.007648
2014-07-01 02:00:00,6693,1,Tuesday,182,27,2,2,0,-0.013788
2014-07-01 03:00:00,4433,1,Tuesday,182,27,3,2,0,-0.008420
2014-07-01 04:00:00,4379,1,Tuesday,182,27,4,2,0,-0.006317


In [13]:
iforest_results[iforest_results['Anomaly'] == 1].head()

,value,day,day_name,day_of_year,week_of_year,hour,is_weekday,Anomaly,Anomaly_Score
timestamp,,,,,,,,,
2014-07-13,50825,13,Sunday,194,28,0,7,1,0.005053
2014-07-27,50407,27,Sunday,208,30,0,7,1,0.006999
2014-08-03,48081,3,Sunday,215,31,0,7,1,0.012103
2014-09-28,53589,28,Sunday,271,39,0,7,1,0.003189
2014-10-05,48472,5,Sunday,278,40,0,7,1,0.001438


In [14]:
import plotly.graph_objects as go

# plot value on y-axis and date on x-axis**
fig = px.line(iforest_results, x=iforest_results.index, y="value", title='NYC TAXI TRIPS - UNSUPERVISED ANOMALY DETECTION', template = 'plotly_dark')

# create list of outlier_dates**
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index

# obtain y value of anomalies to plot**
y_values = [iforest_results.loc[i]['value'] for i in outlier_dates]

fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()